In [1]:
from collections import Counter
import pandas as pd
import warnings

In [2]:
# Suppress warning from openpyxl that the workbook contains no default style:
with warnings.catch_warnings(record=True):
    df = pd.read_excel("Selected_for_analysis.xlsx", na_filter=False)

In [3]:
mapping = {"Strongly disagree": 1, 
           "Somewhat disagree": 2, 
           "Neither agree nor disagree": 3,
           "Somewhat agree": 4,
           "Strongly agree": 5}

def ratings_from_df(i,df):
    key = str(i)
    return [mapping[rating] for rating in df[key] if rating != '']
    

def get_raw_counts(df):
    """
    Get number of times each rating was provided.
    """
    rating_index = {str(i): ratings_from_df(i,df) for i in range(1000)}
    counts = Counter(rating for ratings in rating_index.values() 
                            for rating in ratings)
    return counts


def numbers_above_i(df):
    """
    Get number of times the average rating was equal to or greater than i.
    """
    rating_index = {str(i): ratings_from_df(i,df) for i in range(1000)}
    avg_ratings  = {i : sum(ratings)/len(ratings) for i, ratings in rating_index.items()}
    above_i = {i:0 for i in range(1,6)}
    for rating in avg_ratings.values():
        for i in above_i:
            if rating >= i:
                above_i[i] += 1
    return above_i

In [4]:
raw_counts = get_raw_counts(df)
above_i = numbers_above_i(df)

table = [["Rating"] + [str(i) for i in range(1,6)],
         [r"Raw counts"] + [f"{raw_counts[i]}" for i in range(1,6)],
         [r"\% of ratings"] + [f"{(raw_counts[i]/3000) * 100:.0f}" for i in range(1,6)],
         [r"\% average rating >= rating"] + [f"{(above_i[i]/10):.0f}" for i in range(1,6)]]

for row in table:
    print(' & '.join(row), r'\\')

Rating & 1 & 2 & 3 & 4 & 5 \\
Raw counts & 689 & 571 & 333 & 782 & 625 \\
\% of ratings & 23 & 19 & 11 & 26 & 21 \\
\% average rating >= rating & 100 & 79 & 55 & 32 & 7 \\
